# Enrichment Analysis Visualizer

This appyter creates a variety of visualizations for enrichment analysis results for one selected Enrichr library, and may be run as either a standalone appyter from the [Appyter Catalog](https://appyters.maayanlab.cloud/#/Enrichment_Analysis_Visualizer) or programmatically from the [Enrichr](https://maayanlab.cloud/Enrichr/) results page. 

For simplicity, the only inputs for this appyter are a gene list and one library. Other parameters are set to default values in the cell below. You can download the notebook, change these parameters, and rerun it if you wish.

The pre-processed libraries used to create the scatter plot and hexagonal canvas visualizations can be found [here](https://github.com/MaayanLab/Enrichr-Viz-Appyter/tree/master/Enrichr-Processed-Library-Storage). 

**A link to the full analysis results on the Enrichr website can be found at the bottom of this page.**

In [14]:
# Scatter Plot Imports
from maayanlab_bioinformatics.enrichment import enrich_crisp
import matplotlib as mpl

# Bar Chart Imports
import pandas as pd 
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
import time
from matplotlib.ticker import MaxNLocator
from IPython.display import display, FileLink, HTML, Markdown

# Hexagonal Canvas Imports
import math
import uuid
import urllib
from textwrap import dedent
from string import Template
from operator import itemgetter

# Bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()

Loading BokehJS ...

In [15]:
gene_list_input_factor1 = ['EPHA5', 'EGF', 'EPHA7', 'TEC', 'TEK', 'EPHA2', 'CALM3', 'EPHA4', 'EPHA3', 'LEF1', 'LRP6', 'MST1R', 'NTN1', 'PTPDC1', 'IFNL1', 'MYC', 'BTRC', 'NGFR', 'NDFIP1', 'INSR', 'PRKCD', 'PLK3', 'PTK2', 'FOSL2', 'FOSL1', 'WNK3', 'WNK4', 'PRKD1', 'SDC2', 'PRKCQ', 'TXK', 'NEDD4L', 'BCL10', 'ICAM1', 'MYLK', 'PRKACG', 'SOX2', 'JUNB', 'SOX6', 'PRKACA', 'DUSP6', 'JUN', 'DUSP4', 'DUSP1', 'DUSP2', 'SPHK2', 'KLHL3', 'NFATC2', 'VEGFB', 'CEACAM1', 'KITLG', 'CD28', 'TLR9', 'ARHGEF2', 'HOXD4', 'TLR6', 'SGK3', 'TLR4', 'GUCY2D', 'RET', 'DET1', 'CITED1', 'SDC3', 'SDC4', 'AATK', 'HNF4G', 'ADRB2', 'PTPRJ', 'DUOX1', 'SOCS3', 'STK36', 'STK39', 'NFIL3', 'HAVCR2', 'MEF2A', 'SYK', 'PARP1', 'FOS', 'MAS1', 'ADAM17', 'KIT', 'RFWD2', 'FOSB', 'PPARG', 'TRIP13', 'CD47', 'RPTOR', 'GPNMB', 'NINL', 'MAP3K3', 'PTPN1', 'ABI1', 'CASS4', 'CSPG4', 'PTPN6', 'ITGA2B', 'SLC1A1', 'EGFR', 'FOXO1', 'CDH5', 'CCND1', 'NTF3', 'NTF4', 'GRAP2', 'ADORA1', 'RBBP8', 'SH2B3', 'SMAD3', 'PRNP', 'HGF', 'TIRAP', 'DUPD1', 'NFKB1', 'PGF', 'NR4A2', 'EFNA1', 'DLG1', 'MYD88', 'DUSP21', 'DLG4', 'TRAF7', 'MAFG', 'MAP3K10', 'FGFR1', 'FGF10', 'PFKFB1', 'FGFR2', 'CSF1R', 'IRS1', 'PXN', 'SIRPG', 'FGF1', 'FGF2', 'ETS1', 'ETS2', 'IGF1R', 'FGF5', 'FGF8', 'BAG3', 'SIRPA', 'ABL2', 'ROS1', 'VDR', 'ANGPT4', 'ANGPT2', 'ITGA4', 'ANGPT1', 'BDNF', 'ETV1', 'IRAK4', 'IRF2BP1', 'ETV4', 'DAB1', 'ADORA2A', 'LCP2', 'NFE2L2', 'MAST1', 'BCAR3', 'YWHAG', 'GSK3B', 'YWHAB', 'DUSP18', 'DUSP19', 'CHD3', 'DUSP14', 'IRF2BPL', 'SIRPB1', 'DBP', 'GHR', 'IRAK2', 'GRK5', 'IRAK3', 'IRAK1', 'ERBB3', 'DRD2', 'CASP1', 'NTRK2', 'NTRK3', 'DAPK1', 'TIE1', 'LMTK2', 'TIAM1', 'ANG', 'CRK', 'MET', 'TRIB2', 'TRIB1', 'HLF', 'DRAXIN', 'FLT1', 'FLT3', 'PPM1M', 'CEBPG', 'ITGB3', 'CAMK2B', 'PTEN', 'PIK3CG', 'MALT1', 'SPRED2', 'SPRED1', 'RPS6KA3', 'MAPK3', 'NOS1', 'MAPK1', 'ITGAV', 'FYN', 'MAPK4']
gene_list_input_factor2 = ['SMARCA1', 'FRS2', 'ATG14', 'ATG13', 'SRRM2', 'PML', 'ZFYVE16', 'KATNAL2', 'TEK', 'ASF1A', 'ULK1', 'CDK13', 'ASB6', 'RB1', 'PRKAA2', 'CCNK', 'SATB2', 'ARPC5L', 'PRKAA1', 'CRKL', 'DPYSL2', 'RBM15B', 'DPYSL5', 'CHEK1', 'BSG', 'DPYSL3', 'DPYSL4', 'AKT1', 'TIPIN', 'INSR', 'PRKCB', 'PRKCA', 'BAZ1B', 'ARFGAP2', 'PBX1', 'PTK2', 'BTC', 'TPX2', 'PIK3R1', 'GRB2', 'HOXB9', 'C9ORF72', 'FOXE1', 'PRKDC', 'ELK1', 'ICAM1', 'LARP1', 'SOX2', 'SNRNP70', 'MLST8', 'SYNPO', 'APOB', 'LYN', 'EDN1', 'VEGFC', 'PRPF40A', 'TUBB2A', 'NCOR1', 'CEACAM1', 'MAPT', 'KLHL9', 'ACTR10', 'LIMS1', 'POT1', 'ARPC1A', 'DDR1', 'MTTP', 'SDC4', 'PEA15', 'TGFA', 'ADRB2', 'FXR2', 'BCLAF1', 'GAR1', 'TMED3', 'BRD4', 'SVIL', 'CSNK2A2', 'CSNK2A1', 'RPA2', 'RPA3', 'RRP1B', 'RPA1', 'HCK', 'NFIB', 'CTTN', 'NFIC', 'C21ORF2', 'TIMELESS', 'HBEGF', 'CD44', 'FOXA3', 'FOXA1', 'DPYS', 'PRKAG3', 'RSF1', 'ASH2L', 'TCTN3', 'ENPP1', 'RIOK1', 'AP3S2', 'RICTOR', 'PTPN1', 'SEC24A', 'CAPZA2', 'PPIG', 'ROR1', 'PTPN6', 'CD81', 'AURKA', 'PRKX', 'EGFR', 'ARHGAP44', 'AP3M2', 'RBBP5', 'SH3BP1', 'AP3M1', 'TRAPPC2L', 'TRAPPC3', 'TRAPPC4', 'TRAPPC1', 'NEK5', 'TRAPPC8', 'PRNP', 'HGF', 'TNFRSF10A', 'EEF2', 'REEP5', 'EIF4G1', 'CSF1R', 'PXN', 'CUL5', 'KMT2D', 'RIF1', 'KLHL13', 'RLF', 'DPY30', 'ARHGAP17', 'ETS1', 'IGF1R', 'CHAF1A', 'EEF2K', 'HELB', 'CLSPN', 'SNIP1', 'NEK1', 'EIF4EBP2', 'ZC3H18', 'ABL1', 'CBX5', 'ITGA4', 'CBX3', 'SMARCE1', 'PARVB', 'HMGA1', 'TICRR', 'MERTK', 'THRAP3', 'SMCR8', 'PLEC', 'EIF4E2', 'BCAR1', 'RTN3', 'KDM5C', 'EIF4B', 'GSK3A', 'UBA5', 'CHD8', 'CHD7', 'TARS', 'FOXK1', 'GHR', 'PAGR1', 'DNAJB2', 'P2RY6', 'ERBB2', 'GRK7', 'DVL2', 'ERBB3', 'HIST1H1A', 'EIF4E', 'MDN1', 'NTRK2', 'NTRK3', 'SMARCC2', 'KDM2A', 'SGTB', 'KATNA1', 'NGF', 'MTOR', 'KLHL42', 'TBC1D5', 'PEAK1', 'MET', 'CRK', 'ERCC6', 'BET1', 'ZMYND8', 'FLT1', 'SMARCD3', 'SRC', 'DCTN4', 'RB1CC1', 'AMBRA1', 'SPRED1', 'POLR2A', 'BAHD1', 'TRA2B', 'UBQLN1', 'NCK1', 'NCK2']
gene_list_input_factor3 = ['AC023283.1', 'CDK18', 'PDGFRB', 'PDGFRA', 'ZBTB16', 'OSM', 'ATP11C', 'KLHL25', 'HIPK1', 'ERN1', 'SHTN1', 'OBSCN', 'CD9', 'SPARCL1', 'TNIK', 'EPHA1', 'ROPN1', 'PTPN6', 'EPHA4', 'TP53', 'CNTN1', 'TUBB1', 'ATP10D', 'ATP10B', 'ATP10A', 'NTN1', 'PTPDC1', 'CTGF', 'SULT2A1', 'ATOH8', 'EFNB2', 'EFNB1', 'NTF3', 'NTF4', 'SLAMF6', 'SPP1', 'GBP3', 'GBP2', 'MAP3K5', 'LDLR', 'RUNX1T1', 'MAP3K6', 'GBP4', 'MAP2K3', 'PRKCD', 'SH2D1A', 'SH2D1B', 'ARID5B', 'ESR1', 'DUPD1', 'MASTL', 'PTK2', 'SULT2B1', 'H1F0', 'PIK3R1', 'WNK1', 'GDNF', 'WNK4', 'FGF10', 'FGFR1', 'LHX8', 'CARD16', 'TCF3', 'FBN1', 'FGFR2', 'CSF1R', 'CD274', 'CALCRL', 'ATP8A2', 'ATP8A1', 'PDGFC', 'DMP1', 'PDGFB', 'PDGFA', 'FGF1', 'PKMYT1', 'FGF2', 'CFP', 'FGF4', 'NR4A1', 'C3', 'FGF8', 'GRIN2B', 'TNFSF11', 'PIM2', 'PIM1', 'IL6R', 'MAP2K6', 'MAP2K7', 'DUSP6', 'FIGLA', 'KSR2', 'ITGA5', 'ITGA4', 'STRADB', 'BDNF', 'HMGA2', 'FN1', 'EMP2', 'PTPN11', 'NFATC2', 'PDCD1LG2', 'CDC42BPB', 'LYZ', 'SULT1B1', 'KAT2B', 'TUBB2B', 'CDK5', 'ID3', 'ID2', 'FAM20C', 'CALU', 'SGK1', 'RET', 'DET1', 'EBI3', 'AATK', 'PTPN22', 'DUSP19', 'CD3E', 'DUSP14', 'CYR61', 'SULT1C3', 'DUSP10', 'ERBB3', 'NFIL3', 'DISC1', 'AMELX', 'HAVCR2', 'FCER1G', 'ZNF622', 'HIST1H1B', 'HIST1H1C', 'CR1', 'TMEM30A', 'DAPK1', 'H2AFX', 'STAT3', 'KATNA1', 'ADAM10', 'LMTK2', 'HCK', 'TUBB4A', 'DAB2IP', 'ZAP70', 'CAT', 'KIT', 'STK17B', 'DNAJA3', 'NUCB1', 'CRK', 'TRIB2', 'RAMP1', 'RAMP2', 'TRIB1', 'CD44', 'PPM1M', 'IL27', 'ROPN1B', 'ITGB3', 'WAS', 'TTN', 'RPS6KA1', 'ENC1', 'RPS6KA2', 'MAPK3', 'EPHB1', 'LDLRAP1', 'FYN', 'EPHB3', 'NCK1', 'EPHB4']
gene_list_factor1and3 = ['PTPN1', 'MAP3K3', 'PDGFRB', 'PDGFRA', 'EGF', 'HIPK1', 'TEC', 'SHTN1', 'ABI1', 'TEK', 'SPARCL1', 'TNIK', 'EPHA1', 'PTPN6', 'TP53', 'LEF1', 'NTN1', 'PTPDC1', 'EGFR', 'CTGF', 'CDH5', 'CCND1', 'NTF3', 'NTF4', 'BTRC', 'GBP2', 'MAP3K5', 'SH2B3', 'RUNX1T1', 'MAP3K6', 'NGFR', 'INSR', 'MAP2K3', 'PRKCD', 'SH2D1A', 'SH2D1B', 'TIRAP', 'ESR1', 'DUPD1', 'PTK2', 'FOSL2', 'DLG1', 'MYD88', 'DUSP21', 'H1F0', 'PIK3R1', 'DLG4', 'WNK1', 'WNK4', 'PRKD1', 'FGFR1', 'SDC2', 'TCF3', 'PFKFB1', 'FGFR2', 'IRS1', 'NEDD4L', 'PDGFB', 'PDGFA', 'FGF1', 'FGF2', 'ETS1', 'ETS2', 'IGF1R', 'MYLK', 'NR4A1', 'FGF5', 'PRKACG', 'FGF8', 'GRIN2B', 'SOX2', 'BAG3', 'SIRPA', 'ABL2', 'ROS1', 'JUNB', 'MAP2K6', 'PRKACA', 'DUSP6', 'JUN', 'DUSP4', 'ITGA5', 'ANGPT2', 'DUSP2', 'ITGA4', 'BDNF', 'SPHK2', 'FN1', 'ETV1', 'PTPN11', 'KLHL3', 'NFATC2', 'IRAK4', 'DAB1', 'CDK5', 'ID3', 'FAM20C', 'TLR9', 'LCP2', 'SGK3', 'NFE2L2', 'SGK1', 'RET', 'YWHAG', 'GSK3B', 'DET1', 'YWHAB', 'SDC3', 'SDC4', 'AATK', 'PTPN22', 'DUSP18', 'CHD3', 'PTPRJ', 'DUSP14', 'SOCS3', 'DBP', 'IRAK2', 'IRAK3', 'IRAK1', 'ERBB3', 'CASP1', 'DISC1', 'HAVCR2', 'FCER1G', 'ZNF622', 'HIST1H1B', 'HIST1H1C', 'NTRK2', 'MEF2A', 'NTRK3', 'DAPK1', 'SYK', 'H2AFX', 'STAT3', 'PARP1', 'TIE1', 'FOS', 'ADAM10', 'LMTK2', 'TIAM1', 'HCK', 'ZAP70', 'ADAM17', 'CAT', 'KIT', 'RFWD2', 'STK17B', 'NUCB1', 'PPARG', 'MET', 'CRK', 'TRIB2', 'TRIB1', 'HLF', 'FLT3', 'PPM1M', 'CEBPG', 'ITGB3', 'PTEN', 'WAS', 'TTN', 'RPTOR', 'SPRED1', 'RPS6KA1', 'RPS6KA3', 'RPS6KA2', 'MAPK3', 'EPHB1', 'NOS1', 'MAPK1', 'ITGAV', 'FYN', 'EPHB3', 'NCK1']
enrichr_library = 'BioPlanet_2019'
genes = [x.strip() for x in gene_list_factor1and3]

In [16]:
# Error handling
class NoResults(Exception):
    pass 
class APIFailure(Exception):
    pass

In [17]:
# Enrichr API Function for Manhattan Plot and Bar Chart
# Takes a gene list and Enrichr libraries as input
def Enrichr_API(enrichr_gene_list, all_libraries):

    all_terms = []
    all_pvalues =[] 
    all_adjusted_pvalues = []

    for library_name in all_libraries : 
        ENRICHR_URL = 'https://maayanlab.cloud/Enrichr/addList'
        genes_str = '\n'.join(enrichr_gene_list)
        description = ''
        payload = {
            'list': (None, genes_str),
            'description': (None, description)
        }

        response = requests.post(ENRICHR_URL, files=payload)
        if not response.ok:
            raise APIFailure

        data = json.loads(response.text)
        time.sleep(0.5)
        ENRICHR_URL = 'https://maayanlab.cloud/Enrichr/enrich'
        query_string = '?userListId=%s&backgroundType=%s'
        user_list_id = data['userListId']
        short_id = data["shortId"]
        gene_set_library = library_name
        response = requests.get(
            ENRICHR_URL + query_string % (user_list_id, gene_set_library)
         )
        if not response.ok:
            raise APIFailure

        data = json.loads(response.text)

        if len(data[library_name]) == 0:
            raise NoResults

        short_results_df  = pd.DataFrame(data[library_name][0:10])
        all_terms.append(list(short_results_df[1]))
        all_pvalues.append(list(short_results_df[2]))
        all_adjusted_pvalues.append(list(short_results_df[6]))
        
        results_df  = pd.DataFrame(data[library_name])
        # adds library name to the data frame so the libraries can be distinguished
        results_df['library'] = library_name.replace('_', '')

    return [results_df, short_results_df, all_terms, all_pvalues, all_adjusted_pvalues, str(short_id)]

In [18]:
# Scatter Plot Parameters
significance_value = 0.05

# Bar Chart Parameters
figure_file_format = ['png', 'svg']
output_file_name = 'Enrichr_results_bar'
color = 'lightskyblue'
final_output_file_names = ['{0}.{1}'.format(output_file_name, file_type) for file_type in figure_file_format]

# Hexagonal Canvas Parameters
canvas_color = 'Blue'
num_hex_colored = 10

# Manhattan Plot Parameters
manhattan_colors = ['#003f5c', '#7a5195', '#ef5675', '#ffa600']

In [20]:
def get_library(lib_name):
    '''
    Returns a dictionary mapping each term from the input library to 
    its associated geneset. 
    '''
    raw_lib_data = []

    with urllib.request.urlopen('https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=' + lib_name) as f:
        for line in f.readlines():
            raw_lib_data.append(line.decode("utf-8").split("\t\t"))

    name = []
    gene_list = []
    lib_data = {}

    for i in range(len(raw_lib_data)):
        name += [raw_lib_data[i][0]]
        raw_genes = raw_lib_data[i][1].replace('\t', ' ')
        gene_list += [raw_genes[:-1]]
    
    lib_data = {a[0]:a[1].split(' ') for a in zip(name, gene_list)}
    return lib_data


def volcano_plot(library_name, lib):
    '''
    Make volcano plot of odds ratio vs. significance for input library.
    '''
    enrich_results = enrich_crisp(genes, lib, 21000, True)

    res_df = pd.DataFrame(
        [ [
            term, 
            res.pvalue, 
            res.odds_ratio,
            res
        ] for (term, res) in enrich_results ], 
        columns=['term', 'pvalue', 'odds_ratio', 'res']
    )

    res_df['log_pval'] = np.negative(np.log10(res_df['pvalue']))

    return res_df

In [21]:
lib_data = get_library(enrichr_library)
plot = volcano_plot(enrichr_library, lib_data)
plot.to_csv('../../results/phuego/enrichr_dfs/factor3and1.csv', index=False)